In [3]:
import os, pathlib, sys
from fnmatch import fnmatch
from pathlib import Path
import re
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import tensorflow as tf
import keras
import random
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import util

c:\Users\Thomas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
# Calculate Content Similarity based on the Average Submission for each Subreddit
---

In [5]:
df = pd.read_json('../data/submissions_preprocessed.json')
df['word_embedding'] = np.nan
df

,author,created_utc,permalink,selftext,subreddit,word_embedding
0,jrizos,1314204222,/r/addiction/comments/jt0n7/a_family_member_ha...,This person has been using for roughly 10 year...,addiction,NaN
1,FlashGameAddict,1325584690,/r/addiction/comments/o0y5w/im_quitting_online...,I'm addicted to online games. Most recently I'...,addiction,NaN
2,themarknessmonster,1327220038,/r/addiction/comments/or9wd/is_it_possible_for...,"I think I might be addicted to light, or, more...",addiction,NaN
3,jmc726,1333120496,/r/addiction/comments/rl03q/tramodol_abuse_any...,"For anyone not familiar with Tramadol, it's a ...",addiction,NaN
4,PetiePal,1334264821,/r/addiction/comments/s6swk/my_friend_is_becom...,Hey guys. I've got a friend who recently came ...,addiction,NaN
...,...,...,...,...,...,...
1579814,throw_away_the_panda,1624366207,/r/SelfHate/comments/o5meab/i_genuinely_try_an...,I'm sat at my desk again and I'm unable to kee...,SelfHate,NaN
1579815,twisted-spirit,1624495644,/r/SelfHate/comments/o6pzta/i_feel_like_i_dont...,"The more I try to make friends, or pretend I'm...",SelfHate,NaN
1579816,DedKulak1917,1624593084,/r/SelfHate/comments/o7ft41/really_not_a_fan_o...,Recently cheated on my fiancé. We are in the p...,SelfHate,NaN
1579817,edensrotting,1624980579,/r/SelfHate/comments/oaall2/another_person_her...,"I'm a self destructive person and honestly, i ...",SelfHate,NaN


In [9]:

unique_subreddits = list(df.subreddit.unique())
columns = unique_subreddits
content_similarity_matrix = pd.DataFrame(columns=columns)
save_path = '../data/content_cosine_similarity_matrix.json'

list_of_all_similarities = [] 

for subreddit in unique_subreddits:
    similarities = []
    for subreddit_to_compare in unique_subreddits:
        print('\n---\nNow comparing: ', subreddit)
        print('with: ' + subreddit_to_compare)
        # ---------------------------------------------------------------------------------
        # Special Apply Method to extract the string
        #
        print('Checkpoint: apply')
        source_df = df[df['subreddit'] == subreddit]
        target_df = df[df['subreddit'] == subreddit_to_compare]

        source_df['avg_embedding'] = source_df.word_embedding.apply(lambda x: np.fromstring(x[1:-1], sep=' '))
        target_df['avg_embedding'] = target_df.word_embedding.apply(lambda x: np.fromstring(x[1:-1], sep=' '))
        # ---------------------------------------------------------------------------------
        # String to Numpy Array
        #
        print('Checkpoint: to numpy')
        source_temp = source_df['avg_embedding'].to_numpy()
        target_temp = target_df['avg_embedding'].to_numpy()
        # ---------------------------------------------------------------------------------
        # Numpy Array to List
        #
        print('Checkpoint: to list')
        source_temp = source_temp.tolist()
        target_temp = target_temp.tolist()
        # ---------------------------------------------------------------------------------
        # Numpy Array to Tensor
        #
        print('Checkpoint: to tensor')
        source_avg_embeddings = torch.Tensor(source_temp)
        target_avg_embeddings = torch.Tensor(target_temp)
        source_avg_embeddings = torch.mean(source_avg_embeddings, 1)
        target_avg_embeddings = torch.mean(target_avg_embeddings, 1)
        
        # print('Checkpoint: to tensor')
        # sub_tensors = torch.from_numpy(sub_temp)
        # overlap_tensors = torch.from_numpy(overlap_temp)
        
        # Normalize
        # print('Checkpoint: normalize')
        # sub_tensors = torch.nn.functional.normalize(subreddit_avg_embeddings)
        # overlap_tensors = torch.nn.functional.normalize(overlap_avg_embeddings)
        
        print('Checkpoint: to numpy')
        a1 = source_avg_embeddings.numpy()
        a2 = target_avg_embeddings.numpy()
        
        print(a1.shape)
        # print(a2)
        print(a2.shape)
        
        # Convert to numpy array
        if(len(a1) > len(a2)):
            diff = len(a1) - len(a2)
            print("Case 1")
            a2 = np.pad(a2, (0, len(a1) - len(a2)), 'constant')
            # a2 = np.concatenate([a2, np.zeros(len(a1) - len(a2))])
            # zeros = np.zeros((diff, 0))
            # a2 = np.concatenate((a2[0], zeros))
        else:
            diff = len(a2) - len(a1)
            print("Case 2")
            a1 = np.pad(a1, (0, len(a2) - len(a1)), 'constant')
            # a1 = np.concatenate([a1, np.zeros(len(a2) - len(a1))])
            # zeros = np.zeros((diff, 0))
            # a1 = np.concatenate((a1[0], zeros))
        
        print(a1.shape)
        print(a2.shape)
        
        # Back to Tensor
        mean_embedding1 = torch.from_numpy(a1)
        mean_embedding2 = torch.from_numpy(a2)
        
        # Compute cosine-similarities
        cosine_scores = util.cos_sim(mean_embedding1, mean_embedding2)
        # cosine_scores = cosine_similarity(mean_embedding1, mean_embedding2)
        print("COSINE SCORE of the Average Embeddings:")
        print(cosine_scores)
        similarities.append(cosine_scores)
        print(" Cosine Similarities\n", similarities)
    list_of_all_similarities.append(similarities)
cosine_matrix = pd.DataFrame(list_of_all_similarities, columns=unique_subreddits)
cosine_matrix.index = unique_subreddits
cosine_matrix.to_json(save_path)
print("finished")
display(cosine_matrix)


---
Now comparing:  addiction
with: addiction
Checkpoint: apply


TypeError: 'float' object is not subscriptable